In [1]:
import numpy as np
import pandas as pd

In [2]:
import sent2vec
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from scipy.spatial import distance

In [3]:
model_path = "BioSentVec_PubMed_MIMICIII-bigram_d700.bin"
model = sent2vec.Sent2vecModel()
try:
    model.load_model(model_path)
except Exception as e:
    print(e)
print('model successfully loaded')

model successfully loaded


In [16]:
df=pd.read_csv('episode1_notes_sent.csv')
#print(df.TEXT)
df.head()

,Hours,CATEGORY,DESCRIPTION,TEXT
0,16.158056,Radiology,CHEST (PORTABLE AP),[**2128-5-12**]\n7:27 AM CHEST (PORTABLE AP) ...
1,17.158056,Radiology,CHEST (PORTABLE AP),[**2128-5-12**]\n7:27 AM CHEST (PORTABLE AP) ...


In [17]:

#sentence = preprocess_sentence('Breast cancers with HER2 amplification have a higher risk of CNS metastasis and poorer prognosis.')
# sentence ='Breast cancers with HER2 amplification have a higher risk of CNS metastasis and poorer prognosis.'
# print(sentence)
# sentence=model.embed_sentence(sentence)

#def convert_to_list(df):
    
df.TEXT=df.TEXT.apply(model.embed_sentences)
df.head()
#print(df.TEXT[0].shape)
# sentence= df.TEXT
# print(type(sentence))
# print(sentence.size)
# #print(sentence)
# for i in sentence:
#     sen= i.split('\n')
#     sen=np.asarray(sen)
#     #print(sen)
#     print(type(sen))
#     sentence=model.embed_sentences(sen)
#     print(type(sentence))
#     print(sentence.shape)
# #df.TEXT = df.apply(model.embed_sentence)
#print(df.TEXT)


,Hours,CATEGORY,DESCRIPTION,TEXT
0,16.158056,Radiology,CHEST (PORTABLE AP),"[[-1.639137, 0.90079296, 0.1578581, -0.1033803..."
1,17.158056,Radiology,CHEST (PORTABLE AP),"[[-1.639137, 0.90079296, 0.1578581, -0.1033803..."


In [18]:
print(df.TEXT[0].shape)

(968, 700)


In [29]:
df2=pd.read_csv('episode1_notes_sent.csv')
#print(df2.TEXT[0])
df2.head()
def convert_to_list(i):
    sen= i.split('\n')
    sen=[x for x in sen if len(x)>=3]
    sen=np.asarray(sen)
    return(sen)
df2.TEXT=df2.TEXT.apply(convert_to_list)
print(df2.TEXT[0].shape)
#print(df2.TEXT[0])

(20,)


In [30]:
print(type(df2.TEXT[0]))
df2.TEXT=df2.TEXT.apply(model.embed_sentences)
df2.head()
print(df2.TEXT[0].shape)

<class 'numpy.ndarray'>
(20, 700)


In [80]:
import numpy as np
from torch.utils.data import Dataset
import os
import pickle
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F 
import pandas as pd

class EpisodeDataset(Dataset):
    
    def __init__(self, obs1, obs2, los):
        self.x1 = obs1
        self.x2 = obs2
        self.y = los
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        return (self.x1[index], self.x2[index], self.y[index])
        
x1_train=torch.torch.rand(32,1,16)
x2_train=torch.rand(32,1,16)
Y_train= torch.randint(low=1, high=10, size=(32,1))
train_dataset = EpisodeDataset(x1_train, x2_train, Y_train)
x1_val=torch.rand(32,1,16)
x2_val=torch.rand(32,1,16)
Y_val= torch.randint(low=1, high=10, size=(32,1))
val_dataset = EpisodeDataset(x1_val, x2_val, Y_val)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,shuffle=True) 
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32,shuffle=True) 
# for data in train_dataset:
#     print(data[0].shape, data[1].shape, data[2].shape)
#     break

In [87]:
class EpisiodeCNN(nn.Module):
    def __init__(self):
        super(EpisiodeCNN, self).__init__()
        #input shape 1x2x16
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=2, padding=1, stride = 1)
        16-2*2*1
        #output shape 16x3x17
        #input shape  16x2x16
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=2, padding=1, stride = 1)
        #output shape 32x4x18
        #input shape  32x4x18
        self.pool1 = nn.MaxPool2d(2,2)
        #output shape 32x2x9
        #input shape 32x2x9
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=2, padding=1, stride = 1)
        #output shape 64x3x10
        self.fc1 = nn.Linear(64*3*10, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 1)

    def forward(self, x1,x2):
        #input is of shape (batch_size=32,1,16) if you did the dataloader right
        x=torch.cat((x1, x2), axis=1)
        #Expected x shape =2x16
        x = x.unsqueeze(1)   #1x2x16
#         print("input", x.shape)
        x = F.relu(self.conv1(x))
#         print("conv1", x.shape)
        x = F.relu(self.conv2(x))
#         print("conv2", x.shape)
        x = self.pool1(x)
#         print("pool",x.shape)
        x = F.relu(self.conv3(x))
#         print("conv3", x.shape)
        x = x.view(-1, 64 * 3 * 10)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
#         print("result",x.shape)
        return (x.double())

In [89]:
model = EpisiodeCNN()
learning_rate = 0.01
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr =learning_rate )

def train(model, train_loader, n_epochs):
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x1,x2, y in train_loader:
            optimizer.zero_grad()
            y_hat = model(x1,x2)
#             print("y sizes", y_hat.shape, y.shape)
            loss = criterion(y_hat, y.double())
#             print(loss)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_loss = train_loss / len(train_loader)
            print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))

    
# number of epochs to train the model
n_epochs = 100
train(model, train_loader, n_epochs)

Epoch: 1 	Training Loss: 29.447703
Epoch: 2 	Training Loss: 22.267662
Epoch: 3 	Training Loss: 49.260070
Epoch: 4 	Training Loss: 14.410657
Epoch: 5 	Training Loss: 24.711397
Epoch: 6 	Training Loss: 26.939988
Epoch: 7 	Training Loss: 27.558329
Epoch: 8 	Training Loss: 27.570424
Epoch: 9 	Training Loss: 27.492579
Epoch: 10 	Training Loss: 27.205755
Epoch: 11 	Training Loss: 26.669560
Epoch: 12 	Training Loss: 25.796866
Epoch: 13 	Training Loss: 24.074563
Epoch: 14 	Training Loss: 21.158574
Epoch: 15 	Training Loss: 16.299120
Epoch: 16 	Training Loss: 11.379762
Epoch: 17 	Training Loss: 7.612090
Epoch: 18 	Training Loss: 12.491028
Epoch: 19 	Training Loss: 15.707476
Epoch: 20 	Training Loss: 5.782828
Epoch: 21 	Training Loss: 8.207853
Epoch: 22 	Training Loss: 10.845910
Epoch: 23 	Training Loss: 9.600374
Epoch: 24 	Training Loss: 11.293599
Epoch: 25 	Training Loss: 9.601344
Epoch: 26 	Training Loss: 6.897955
Epoch: 27 	Training Loss: 6.679342
Epoch: 28 	Training Loss: 9.437575
Epoch: 29